<a href="https://colab.research.google.com/github/slvannd/2009106120_Silvia_Ananda/blob/main/Copy_of_Gacoan_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas nltk sastrawi scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Baris ini untuk men-download komponen NLTK yang dibutuhkan
# Cukup dijalankan sekali

try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt_tab', quiet=True)

In [ ]:
# --- 1. PEMUATAN DAN PEMBERSIHAN DATA ---
print("### 1. Memuat dan Membersihkan Data Awal ###")
df = pd.read_csv('google.csv')

# Mengganti nama kolom agar mudah dibaca
df = df.rename(columns={'wiI7pd': 'ulasan', 'RfDO5c': 'rating'})
df = df[['ulasan', 'rating']]

# Membersihkan data yang kosong atau tidak valid
df.dropna(subset=['ulasan', 'rating'], inplace=True)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df.dropna(subset=['rating'], inplace=True)
df['rating'] = df['rating'].astype(int)
print("Data awal berhasil dimuat dan dibersihkan.")
print(df.head())

### 1. Memuat dan Membersihkan Data Awal ###


FileNotFoundError: [Errno 2] No such file or directory: 'google.csv'

In [ ]:
print("Jumlah rating untuk setiap jenis rating:")
display(df['rating'].value_counts())

In [ ]:
# --- 2. PELABELAN SENTIMEN ---
print("\n### 2. Memberi Label Sentimen Berdasarkan Rating ###")
def label_sentimen(row):
    if row['rating'] > 3: return 'positif'
    elif row['rating'] < 3: return 'negatif'
    else: return 'netral'
df['sentimen'] = df.apply(label_sentimen, axis=1)

# Baris di bawah ini dikomentari agar data netral tetap dipertahankan
# if len(df[df.sentimen == 'netral']) < 10:
#     df = df[df.sentimen != 'netral']
#     print(f"Info: Kelas 'netral' dihapus karena jumlahnya tidak signifikan. ({len(df[df.sentimen == 'netral'])})")
print("Pelabelan selesai. Jumlah data per sentimen:")
print(df['sentimen'].value_counts())

In [ ]:
# --- 2. PELABELAN SENTIMEN ---
print("\n### 2. Memberi Label Sentimen Berdasarkan Rating ###")
def label_sentimen(row):
    if row['rating'] > 3: return 'positif'
    elif row['rating'] < 3: return 'negatif'
    else: return 'netral'
df['sentimen'] = df.apply(label_sentimen, axis=1)

# Menghapus kondisi untuk menghapus kelas 'netral' agar semua data netral tetap dipertahankan
# if len(df[df.sentimen == 'netral']) < 10:
#     df = df[df.sentimen != 'netral']
#     print(f"Info: Kelas 'netral' dihapus karena jumlahnya tidak signifikan. ({len(df[df.sentimen == 'netral'])})")
print("Pelabelan selesai. Jumlah data per sentimen:")
print(df['sentimen'].value_counts())

In [ ]:
# --- 3. PREPROCESSING TEKS ---
print("\n### 3. Memproses Teks Ulasan (Stemming, dll.) ###")
df['ulasan_bersih'] = df['ulasan'].str.lower()
df['ulasan_bersih'] = df['ulasan_bersih'].apply(lambda x: re.sub(r'[^a-z\s]', '', x))
df['ulasan_bersih'] = df['ulasan_bersih'].apply(word_tokenize)
list_stopwords = set(stopwords.words('indonesian'))
df['ulasan_bersih'] = df['ulasan_bersih'].apply(lambda x: [w for w in x if w not in list_stopwords])
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df['ulasan_bersih'] = df['ulasan_bersih'].apply(lambda x: ' '.join([stemmer.stem(w) for w in x]))
df = df[df['ulasan_bersih'].str.len() > 0]
print("Preprocessing teks selesai.")
print("Contoh hasil:")
print(df[['ulasan', 'ulasan_bersih']].head())

In [ ]:
# --- 4. EKSTRAKSI FITUR & PEMBAGIAN DATA ---
X = df['ulasan_bersih']
y = df['sentimen']
vectorizer = TfidfVectorizer(max_features=1000) # Ambil 1000 kata paling penting
X_tfidf = vectorizer.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42, stratify=y)
print(f"\n### 4. Data Siap Dimodelkan ###\nData Latih: {X_train.shape[0]} ulasan\nData Uji: {X_test.shape[0]} ulasan")

In [ ]:
# --- 5. PEMODELAN DENGAN K-NN ---
print("\n### 5. Mencari Nilai 'K' Terbaik untuk Model ###")
akurasi_terbaik = 0
k_terbaik = 0
# Batas atas K tidak boleh melebihi jumlah data latih
max_k = min(15, X_train.shape[0] - 1)
for k in range(3, max_k + 1, 2):
    knn_temp = KNeighborsClassifier(n_neighbors=k)
    knn_temp.fit(X_train, y_train)
    akurasi = knn_temp.score(X_test, y_test)
    if akurasi > akurasi_terbaik:
        akurasi_terbaik = akurasi
        k_terbaik = k
print(f"Nilai K terbaik ditemukan: {k_terbaik}")

In [ ]:
# --- 6. EVALUASI MODEL FINAL ---
print("\n===== HASIL AKHIR EVALUASI MODEL K-NN =====")
knn = KNeighborsClassifier(n_neighbors=k_terbaik)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

labels = sorted(y.unique())
print(f"\nAkurasi Model: {accuracy_score(y_test, y_pred):.2%}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, labels=labels))
print("Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(y_test, y_pred, labels=labels), index=labels, columns=labels))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred, labels=labels), annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Prediksi Label')
plt.ylabel('Label Sebenarnya')
plt.title('Confusion Matrix (Heatmap)')
plt.show()

Grafik di atas memvisualisasikan bagaimana akurasi model K-NN berubah seiring dengan perubahan nilai K (jumlah tetangga terdekat) dan untuk berbagai rasio pembagian data uji (`test_size`).

Setiap garis berwarna mewakili akurasi model untuk satu `test_size` tertentu.

*   **Sumbu X** menunjukkan nilai K yang dievaluasi (1, 3, 5, ..., 15).
*   **Sumbu Y** menunjukkan nilai akurasi model.
*   **Legenda** menunjukkan `test_size` yang berbeda.

# Task
Evaluate the K-NN model's performance by varying the `test_size` parameter from 0.1 to 0.5. For each `test_size`, train the model using the previously determined optimal `k_terbaik` and record the accuracy. Finally, visualize the relationship between `test_size` and model accuracy using a line plot, and summarize the findings regarding the impact of data split ratios on model performance.

## Definisikan Rentang Rasio Pembagi Data

### Subtask:
Menentukan serangkaian nilai untuk `test_size` (misalnya, dari 0.1 hingga 0.5) yang akan dieksplorasi untuk melihat dampaknya pada performa model.


**Reasoning**:
I need to define a list of `test_size` values as instructed. Using `np.arange` is suitable for generating a sequence of floats.



In [ ]:
import numpy as np
test_sizes = np.arange(0.1, 0.51, 0.1).tolist()
print(f"Rentang test_size yang akan dievaluasi: {test_sizes}")

**Reasoning**:
Now that the `test_sizes` array is defined, I will iterate through each `test_size` value, split the data, train the K-NN model using the optimal `k_terbaik` determined earlier, and then calculate and store the accuracy for each split.



In [ ]:
accuracies = []
for test_size in test_sizes:
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=test_size, random_state=42, stratify=y)

    # Re-instantiate and train the model with k_terbaik
    knn = KNeighborsClassifier(n_neighbors=k_terbaik)
    knn.fit(X_train, y_train)

    # Calculate accuracy
    accuracy = knn.score(X_test, y_test)
    accuracies.append(accuracy)
    print(f"Test Size: {test_size:.1f}, Accuracy: {accuracy:.2%}")

# Store results in a DataFrame for easier visualization
results_df = pd.DataFrame({'test_size': test_sizes, 'accuracy': accuracies})
print("\nAccuracy results for different test sizes:")
print(results_df)

**Reasoning**:
The previous step successfully calculated the model accuracies for various `test_size` values. Now, to fulfill the task's requirement, I need to visualize this relationship using a line plot. This will clearly show how changing the `test_size` impacts the model's accuracy.



In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(x='test_size', y='accuracy', data=results_df, marker='o')
plt.title('Dampak Test Size terhadap Akurasi Model K-NN')
plt.xlabel('Ukuran Data Uji (Test Size)')
plt.ylabel('Akurasi')
plt.grid(True)
plt.xticks(test_sizes) # Ensure all test_sizes are shown as ticks
plt.ylim(0.95, 1.01) # Set y-axis limits to better visualize accuracy differences
plt.show()

print("Visualisasi hubungan antara test_size dan akurasi model telah selesai.")

### Ringkasan Dampak Ukuran Data Uji pada Performa Model

Dari eksperimen yang dilakukan dengan memvariasikan `test_size`:

*   **Akurasi Tinggi untuk `test_size` Kecil (0.1, 0.2)**: Model mencapai akurasi 100% ketika ukuran data uji relatif kecil (0.1 dan 0.2). Ini menunjukkan bahwa dengan data uji yang lebih sedikit, model cenderung menunjukkan performa yang sangat baik. Namun, ini juga bisa mengindikasikan bahwa data uji mungkin tidak cukup representatif atau memiliki variasi yang terbatas.

*   **Penurunan Akurasi dengan Peningkatan `test_size`**: Ketika `test_size` meningkat menjadi 0.3, 0.4, dan 0.5, akurasi sedikit menurun namun tetap berada pada tingkat yang sangat tinggi (sekitar 97.8% hingga 98.7%). Penurunan ini normal karena model memiliki lebih sedikit data untuk dilatih dan lebih banyak data yang belum terlihat untuk diuji, yang bisa mengungkap generalisasi model yang sebenarnya.

*   **Kecenderungan Stabil**: Meskipun ada sedikit variasi, akurasi model tetap sangat stabil dan tinggi di seluruh rentang `test_size` yang dievaluasi. Ini menunjukkan bahwa model K-NN cukup robust terhadap perubahan rasio pembagian data dalam konteks dataset ini.

**Kesimpulan**: Untuk dataset ini, model K-NN menunjukkan performa yang sangat baik dan konsisten di berbagai rasio `test_size`. Namun, penting untuk dicatat bahwa akurasi yang mendekati sempurna bisa menjadi indikasi potensi masalah seperti *data leakage* atau *imbalanced dataset* yang ekstrem, terutama jika kelas 'negatif' memiliki jumlah sampel yang sangat sedikit seperti yang terlihat pada output `df['sentimen'].value_counts()`. Dalam kasus ini, hanya ada 2 sampel negatif dibandingkan 153 sampel positif, yang bisa membuat model terlihat sangat akurat pada kelas mayoritas.

### Ringkasan Dampak Ukuran Data Uji pada Performa Model

Dari eksperimen yang dilakukan dengan memvariasikan `test_size`:

*   **Akurasi Tinggi untuk `test_size` Kecil (0.1, 0.2)**: Model mencapai akurasi 100% ketika ukuran data uji relatif kecil (0.1 dan 0.2). Ini menunjukkan bahwa dengan data uji yang lebih sedikit, model cenderung menunjukkan performa yang sangat baik. Namun, ini juga bisa mengindikasikan bahwa data uji mungkin tidak cukup representatif atau memiliki variasi yang terbatas.

*   **Penurunan Akurasi dengan Peningkatan `test_size`**: Ketika `test_size` meningkat menjadi 0.3, 0.4, dan 0.5, akurasi sedikit menurun namun tetap berada pada tingkat yang sangat tinggi (sekitar 97.8% hingga 98.7%). Penurunan ini normal karena model memiliki lebih sedikit data untuk dilatih dan lebih banyak data yang belum terlihat untuk diuji, yang bisa mengungkap generalisasi model yang sebenarnya.

*   **Kecenderungan Stabil**: Meskipun ada sedikit variasi, akurasi model tetap sangat stabil dan tinggi di seluruh rentang `test_size` yang dievaluasi. Ini menunjukkan bahwa model K-NN cukup robust terhadap perubahan rasio pembagian data dalam konteks dataset ini.

**Kesimpulan**: Untuk dataset ini, model K-NN menunjukkan performa yang sangat baik dan konsisten di berbagai rasio `test_size`. Namun, penting untuk dicatat bahwa akurasi yang mendekati sempurna bisa menjadi indikasi potensi masalah seperti *data leakage* atau *imbalanced dataset* yang ekstrem, terutama jika kelas 'negatif' memiliki jumlah sampel yang sangat sedikit seperti yang terlihat pada output `df['sentimen'].value_counts()`. Dalam kasus ini, hanya ada 2 sampel negatif dibandingkan 153 sampel positif, yang bisa membuat model terlihat sangat akurat pada kelas mayoritas.

## Final Task

### Subtask:
Meringkas hasil analisis mengenai pengaruh rasio pembagi data terhadap akurasi model, dan mengidentifikasi rasio yang mungkin paling efektif.


## Summary:

### Q&A
*   **Which data split ratio might be most effective?**
    The K-NN model achieved 100% accuracy with `test_size` values of 0.1 and 0.2. While these ratios showed perfect accuracy, it's important to consider the potential for limited representativeness or variation in smaller test datasets. For larger test sizes (0.3, 0.4, 0.5), accuracy remained very high (97.83% to 98.67%), suggesting the model is robust across these splits.

### Data Analysis Key Findings
*   The K-NN model achieved perfect accuracy (100.00%) when the `test_size` was 0.1 and 0.2.
*   As the `test_size` increased to 0.3, 0.4, and 0.5, the model's accuracy slightly decreased but remained very high, ranging from 97.83% to 98.67%.
*   The model demonstrates consistent and high performance across the evaluated `test_size` range (0.1 to 0.5), indicating robustness to changes in data split ratios for this dataset.
*   A significant class imbalance was noted (2 negative samples vs. 153 positive samples), which could contribute to the seemingly perfect accuracy, potentially masking underlying generalization issues, especially if the model primarily performs well on the majority class.

### Insights or Next Steps
*   Investigate the impact of the severe class imbalance on model performance. Consider employing techniques such as oversampling the minority class, undersampling the majority class, or using evaluation metrics less sensitive to imbalance (e.g., F1-score, precision, recall) to get a more accurate understanding of the model's true generalization ability.
*   Given the high accuracy observed even with larger test sets, further testing on a more diverse and balanced dataset would be beneficial to confirm the model's robustness and generalization capabilities beyond the current dataset's specific characteristics.


# Task
Systematically evaluate the K-NN model's accuracy across various K values and test sizes, visualize the results, and summarize the combined impact of these parameters on model performance.

## Define K Values for Exploration

### Subtask:
Specify a range of K values to test for the K-NN model.


**Reasoning**:
I need to generate a list of odd integers for 'K' values, ranging from 1 to 15. A list comprehension or `np.arange` with a step of 2 can be used for this.



In [ ]:
k_values = list(range(1, 16, 2))
print(f"K values to be evaluated: {k_values}")

**Reasoning**:
I need to systematically evaluate the K-NN model's accuracy for each combination of `k_values` and `test_sizes`. I will use nested loops to iterate through these parameters, train the K-NN model, and record the accuracy for each combination. The results will be stored in a DataFrame for further analysis and visualization.



In [ ]:
results = []

for test_size in test_sizes:
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=test_size, random_state=42, stratify=y)

    for k in k_values:
        # Ensure k is not larger than the number of samples in the training set
        if k > X_train.shape[0]:
            print(f"Skipping k={k} for test_size={test_size} as k is larger than training samples ({X_train.shape[0]})")
            continue

        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        accuracy = knn.score(X_test, y_test)
        results.append({'test_size': test_size, 'k_value': k, 'accuracy': accuracy})
        print(f"Test Size: {test_size:.1f}, K: {k}, Accuracy: {accuracy:.2%}")

# Convert results to DataFrame
combined_results_df = pd.DataFrame(results)
print("\nCombined accuracy results:")
print(combined_results_df.head())


**Reasoning**:
I have collected the accuracy results for all combinations of `test_size` and `k_value` in `combined_results_df`. To visualize the combined impact of these parameters on model performance, a heatmap is an effective choice.



In [ ]:
plt.figure(figsize=(10, 6))
heatmap_data = combined_results_df.pivot(index='k_value', columns='test_size', values='accuracy')
sns.heatmap(heatmap_data, annot=True, fmt=".2%", cmap="Blues", cbar_kws={'label': 'Accuracy'})
plt.title('K-NN Model Accuracy across K values and Test Sizes')
plt.xlabel('Test Size')
plt.ylabel('K Value (n_neighbors)')
plt.show()

print("Heatmap visualizing the impact of K values and Test Sizes on model accuracy has been generated.")

In [ ]:
plt.figure(figsize=(12, 7))
sns.lineplot(data=combined_results_df, x='k_value', y='accuracy', hue='test_size', marker='o', palette='viridis')
plt.title('Akurasi Model K-NN vs. Nilai K untuk Berbagai Ukuran Data Uji')
plt.xlabel('Nilai K (n_neighbors)')
plt.ylabel('Akurasi')
plt.xticks(k_values)
plt.ylim(0.90, 1.01)
plt.grid(True)
plt.legend(title='Ukuran Data Uji')
plt.show()

print("Grafik garis Akurasi vs. Nilai K untuk Ukuran Data Uji yang berbeda telah dibuat.")

In [ ]:
print("Jumlah rating untuk setiap jenis rating:")
display(df['rating'].value_counts())

In [ ]:
import matplotlib.pyplot as plt

sentimen_counts = df['sentimen'].value_counts()

plt.figure(figsize=(8, 8))
plt.pie(sentimen_counts, labels=sentimen_counts.index, autopct='%1.1f%%', startangle=90, colors=['#66b3ff','#99ff99','#ffcc99'])
plt.title('Distribusi Sentimen Ulasan')
plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

print("Diagram lingkaran distribusi sentimen telah dibuat.")

Diagram lingkaran di atas menampilkan proporsi masing-masing kategori sentimen (positif, negatif) dalam dataset.

*   Setiap irisan pada diagram lingkaran merepresentasikan satu kategori sentimen.
*   Ukuran irisan menunjukkan persentase atau bagian dari total ulasan yang termasuk dalam sentimen tersebut.
*   Angka persentase pada setiap irisan menunjukkan kontribusi sentimen terhadap keseluruhan data.

### Ringkasan Dampak Gabungan Nilai K dan Ukuran Data Uji pada Performa Model

Dari analisis heatmap yang menampilkan akurasi model K-NN untuk kombinasi nilai K dan ukuran data uji yang berbeda, dapat disimpulkan beberapa hal:

*   **Sensitivitas terhadap Nilai K:**
    *   Pada `test_size` 0.1 dan 0.2, akurasi model mencapai 100% untuk semua nilai K dari 3 hingga 15. Ini menunjukkan bahwa untuk porsi data uji yang kecil, model K-NN sangat stabil dan performanya tinggi, meskipun K=1 sedikit lebih rendah.
    *   Untuk `test_size` yang lebih besar (0.3, 0.4, 0.5), akurasi model cenderung stabil di sekitar 97.8% hingga 98.7% di seluruh rentang nilai K yang diuji (1 hingga 15). Tidak ada perubahan signifikan dalam akurasi saat K bervariasi dalam rentang ini, menunjukkan bahwa dalam kasus ini, model tidak terlalu sensitif terhadap pilihan K setelah K=1.

*   **Sensitivitas terhadap Ukuran Data Uji (Test Size):**
    *   Akurasi tertinggi (100%) terlihat pada `test_size` 0.1 dan 0.2. Ini mungkin karena data uji yang lebih kecil dan kemungkinan kurang bervariasi, atau karena *class imbalance* yang kuat memungkinkan model mencapai akurasi tinggi pada kelas mayoritas.
    *   Seiring peningkatan `test_size` (0.3, 0.4, 0.5), akurasi sedikit menurun tetapi tetap sangat tinggi. Ini adalah perilaku yang diharapkan karena model harus menggeneralisasi ke set data uji yang lebih besar dan mungkin lebih representatif.

*   **Kombinasi Optimal:**
    *   Berdasarkan hasil, `k_terbaik` sebelumnya (K=3) masih menunjukkan performa yang sangat baik, seringkali menghasilkan akurasi 100% untuk `test_size` kecil dan tetap kompetitif untuk `test_size` yang lebih besar. Tidak ada nilai K tunggal yang secara signifikan lebih baik daripada yang lain untuk `test_size` > 0.2.
    *   Jika tujuan adalah untuk mendapatkan akurasi setinggi mungkin (berdasarkan metrik akurasi sederhana ini), maka `test_size` 0.1 atau 0.2 dengan K=3 (atau K ganjil lainnya hingga 15) akan memberikan hasil 100% pada dataset ini.

**Kesimpulan Tambahan:**
Perlu diingat bahwa akurasi yang sangat tinggi (terutama 100%) harus diinterpretasikan dengan hati-hati, terutama mengingat adanya ketidakseimbangan kelas yang signifikan (`negatif`: 2 sampel, `positif`: 153 sampel). Model mungkin hanya 'belajar' untuk memprediksi kelas mayoritas dengan baik. Evaluasi lebih lanjut menggunakan metrik lain seperti F1-score, presisi, atau recall untuk masing-masing kelas akan memberikan gambaran yang lebih akurat tentang kinerja model, terutama untuk kelas minoritas. Model K-NN menunjukkan stabilitas dan kinerja yang kuat pada dataset ini di berbagai rasio pembagian data dan nilai K yang diuji.

## Final Task

### Subtask:
Summarize the findings regarding the relationship between K value, test size, and model accuracy.


## Summary:

### Q&A
The findings regarding the relationship between K value, test size, and model accuracy are summarized as follows:
*   For small test sizes (0.1 and 0.2), the model achieved 100% accuracy for K values ranging from 3 to 15. K=1 showed slightly lower accuracy for these small test sizes.
*   For larger test sizes (0.3, 0.4, and 0.5), the model's accuracy remained consistently high, ranging from 97.83% to 98.67% across all K values from 1 to 15. In these cases, the model showed little sensitivity to the specific K value chosen.
*   Overall, the model demonstrated high stability and performance across the tested K values and test sizes, with smaller test sizes generally yielding higher reported accuracies.

### Data Analysis Key Findings
*   The K-NN model was evaluated across K values from 1 to 15 (odd numbers) and test sizes of 0.1, 0.2, 0.3, 0.4, and 0.5.
*   For `test_size` 0.1 and 0.2, the model achieved 100% accuracy with K values between 3 and 15.
*   For larger `test_sizes` (0.3, 0.4, 0.5), accuracy remained very high, typically between 97.83% and 98.67%, and was largely stable across the tested K values.
*   The heatmap visualization effectively illustrated that smaller test sizes and K values greater than 1 tend to yield the highest accuracies, while larger test sizes show robust performance regardless of K.

### Insights or Next Steps
*   The reported 100% accuracy, especially with smaller test sizes, should be interpreted cautiously due to the significant class imbalance present in the dataset (2 negative samples vs. 153 positive samples).
*   Future analysis should incorporate additional evaluation metrics such as F1-score, precision, and recall, particularly for the minority class, to gain a more comprehensive understanding of the model's true performance and generalization ability.
